In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Flatten, Reshape, Conv2D, Conv2DTranspose

from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
import pandas as pd

# Paths to your data
image_folder = '/content/drive/MyDrive/ML final project/datasets/CLASSES_400_300_Part2/'
csv_path = '/content/drive/MyDrive/ML final project/datasets/iris_labelsShort.csv'

# Load the CSV file into a DataFrame
labels_df = pd.read_csv(csv_path)

# Display the first few rows to verify
print(labels_df.head())

# Use the correct column names from your CSV file
image_names = labels_df['filename']
labels = labels_df['label']

# Load and preprocess images
image_data = []
image_labels = []

for i, image_name in enumerate(image_names):
    image_path = os.path.join(image_folder, image_name)

    # Check if the image exists
    if os.path.exists(image_path):
        # Load the image, resize to 28x28 (or your desired size), and convert to array
        img = load_img(image_path, target_size=(28, 28))  # Resize to 28x28
        img_array = img_to_array(img)  # Convert to array
        img_array = img_array.astype("float32") / 255.0  # Normalize to [0, 1]

        # Append to lists
        image_data.append(img_array)
        image_labels.append(labels[i])  # Append corresponding label

# Convert to NumPy arrays
image_data = np.array(image_data)
image_labels = np.array(image_labels)

print(f"Image data shape: {image_data.shape}")
print(f"Labels shape: {image_labels.shape}")


           filename  label
0   C261_S1_I1.tiff  brown
1  C261_S1_I10.tiff  brown
2  C261_S1_I11.tiff  brown
3  C261_S1_I12.tiff  brown
4  C261_S1_I13.tiff  brown


KeyboardInterrupt: 

In [ ]:
# Define the dimensions of the latent space
latent_dim = 64  # Size of the compressed representation

# Input shape
input_shape = (28, 28, 3)  # Adjust this to match the size of your eye images

# Build the Encoder
input_img = Input(shape=input_shape, name="input_image")
x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = Flatten()(x)
latent_vector = Dense(latent_dim, activation='relu', name="latent_vector")(x)

# Build the Decoder
x = Dense(28 * 28 * 64, activation='relu')(latent_vector)
x = Reshape((28, 28, 64))(x)
x = Conv2DTranspose(64, (3, 3), activation='relu', padding='same')(x)
x = Conv2DTranspose(32, (3, 3), activation='relu', padding='same')(x)
output_img = Conv2DTranspose(3, (3, 3), activation='sigmoid', padding='same', name="output_image")(x)

# Define the Autoencoder Model
autoencoder = Model(inputs=input_img, outputs=output_img, name="autoencoder")

# Compile the Model
autoencoder.compile(optimizer='adam', loss='mse')

# Display the Model Summary
autoencoder.summary()


In [ ]:
# Define the Autoencoder Model
autoencoder = Model(inputs=input_img, outputs=output_img, name="autoencoder")

# Compile the Model
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

# Display the Model Summary
autoencoder.summary()



In [ ]:
#take two images get the vectors of them and then average them and output them to see what you get out
# Train the Autoencoder with a binary cross-entropy-like loss
history = autoencoder.fit(
    image_data,  # Input images
    image_data,  # Target images (same as input for autoencoder)
    epochs=20,   # Number of epochs
    batch_size=32,  # Adjust based on your memory constraints
    validation_split=0.2,  # Use 20% of the data for validation
    shuffle=True
)

# Plot Training and Validation Loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title("Autoencoder Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.show()
